In [1]:
!pip install datasets
!pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 365 kB 24.6 MB/s 
     |████████████████████████████████| 212 kB 71.2 MB/s 
     |████████████████████████████████| 115 kB 70.8 MB/s 
     |████████████████████████████████| 141 kB 2.9 MB/s 
     |████████████████████████████████| 101 kB 11.9 MB/s 
     |████████████████████████████████| 596 kB 69.4 MB/s 
     |████████████████████████████████| 127 kB 33.6 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 60 kB 7.2 MB/s 
     |███████████████████████████

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [3]:
!git clone https://github.com/drehero/geneval

Cloning into 'geneval'...
remote: Enumerating objects: 264, done.
remote: Counting objects: 100% (59/59), done.
remote: Compressing objects: 100% (44/44), done.
remote: Total 264 (delta 15), reused 53 (delta 11), pack-reused 205
Receiving objects: 100% (264/264), 11.11 MiB | 2.58 MiB/s, done.
Resolving deltas: 100% (89/89), done.


In [4]:
import pathlib

import datasets
import numpy as np
import pandas as pd

from geneval.geneval.data.wmt import WMT18
#from geneval.geneval.utils import kendall_score

## MoverScore

In [5]:
from geneval.geneval.metrics.MoverScore.MoverScore import MoverScore as mover

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
refs = [['The dog bit the man.', 'It was not unexpected.', 'The man bit him first.'],
        ['The dog had bit the man.', 'No one was surprised.', 'The man had bitten the dog.']]
sys = ['The dog bit the man.', "It wasn't surprising.", 'The man had just bitten him.']

In [8]:
#mover_score = datasets.load_metric("MoverScore")

mover()._compute(sys,refs)

0.7390777091917533

In [9]:
# kendall score
# https://github.com/Tiiiger/bert_score/blob/master/reproduce/get_wmt18_seg_results.py#L53

def kendall_score(scores_better, scores_worse):
    total = len(scores_better)
    correct = np.sum(np.array(scores_better) > np.array(scores_worse))
    incorrect = total - correct
    return (correct - incorrect)/total

In [10]:
class config:
    metric_name = "MoverScore"
    metric_path = "MoverScore"
    model_type = "bert-base-uncased"

    out_path = pathlib.Path("/content/drive/MyDrive/results/wmt18")

    #target_lang = "en"
    lang_pairs = ["cs-en"] #, "de-en", "et-en", "fi-en", "ru-en", "tr-en", "zh-en"

### Test MoverScore metric on WMT18

In [45]:
#Load WMT18
wmt = WMT18("cs-en", root="/tmp")

In [46]:
# Get 3 examples each
wmt18_ref = wmt.references[:3]
wmt18_worse = wmt.translations_worse[:3]
wmt18_better = wmt.translations_better[:3]
print(wmt18_ref, '\n', wmt18_worse, '\n', wmt18_better)

['The lead roles - hard-boiled Gregor and idealistic Iosef - are played by Florin Piersic Jr. and Corneliu Ulici, their dialogue dubbed in English by Channing Tatum (also an executive producer) and Joseph Gordon-Levitt.', 'The lead roles - hard-boiled Gregor and idealistic Iosef - are played by Florin Piersic Jr. and Corneliu Ulici, their dialogue dubbed in English by Channing Tatum (also an executive producer) and Joseph Gordon-Levitt.', 'The lead roles - hard-boiled Gregor and idealistic Iosef - are played by Florin Piersic Jr. and Corneliu Ulici, their dialogue dubbed in English by Channing Tatum (also an executive producer) and Joseph Gordon-Levitt.'] 
 ['The starred - a hard-rising Margaret and a idealist Iosef - be played by Florin Piersic Jr. and Corneliu Ulici, their screening of Channing Tatum (held a detailed producer) and Joseph Gordon-Levitt on languages synchronised languages.', 'The main characters - a hard-nosed Gregor and an idealistic Iosef - are played by Florin Piers

The 'de-en' language pair has over 77811 sentences. I stopped the reproduction after 3h20 min for this language pair. The reproduction for 'cs-en' is comparatively quite fast (only ~5100 sentences)

In [48]:
#MoverScore for translations_better and the references(in brackets!)
mover()._compute(wmt18_better,[wmt18_ref])

0.7533389882346274

In [49]:
#MoverScore for translations_better and the references(in brackets!)
mover()._compute(wmt18_worse,[wmt18_ref])

0.6775284745279535

In [50]:
wmt.references[1]

'The lead roles - hard-boiled Gregor and idealistic Iosef - are played by Florin Piersic Jr. and Corneliu Ulici, their dialogue dubbed in English by Channing Tatum (also an executive producer) and Joseph Gordon-Levitt.'

Testing the evaluation method for one sentence -> it works

In [51]:
mover_results = {}
#custom script for only 10 examples
wmt = WMT18("cs-en", root="/tmp")

# compute scores
scores_better = mover()._compute(
predictions=wmt.translations_better[1],
references=wmt.references[1]
)
print(scores_better)
scores_worse = mover()._compute(
predictions=wmt.translations_worse[1],
references=wmt.references[1]
)
print(scores_worse)
#ks = kendall_score(scores_better, scores_worse)
#mover_results[lang_pair] = ks



0.6601425996122621
0.52935683972945


Reproduction for only 50 sentences -> it works

In [53]:
mover_results = {}
better_scores = []
worse_scores = []
for lang_pair in ["de-en"]:
    # load data
    wmt = WMT18(lang_pair, root="/tmp")

    #for idx in range(len(wmt.translations_better)):
    for idx in range(50):
    # compute scores
        scores_better = mover()._compute(
            predictions = wmt.translations_better[idx],
            references = wmt.references[idx]
        )
        better_scores.append(scores_better)

        scores_worse = mover()._compute(
            predictions = wmt.translations_worse[idx],
            references = wmt.references[idx]
        )

        worse_scores.append(scores_worse)

    ks = kendall_score(better_scores, worse_scores)
    mover_results[lang_pair] = ks

In [54]:
print(better_scores)
print('\n', worse_scores)
print('\n', mover_results)

[0.7648566038659733, 0.7303037569719356, 0.7648566038659733, 0.7648566038659733, 0.8078728217091813, 0.7303037569719356, 0.7697224643119873, 0.7940094351724705, 0.7697912433418549, 0.7697912433418549, 0.7697912433418549, 0.7697912433418549, 0.7697912433418549, 0.8078728217091813, 0.8078728217091813, 0.7755843039166965, 0.7755843039166965, 0.7755843039166965, 0.7303037569719356, 0.7303037569719356, 0.7697224643119873, 0.7940094351724705, 0.7303037569719356, 0.7697224643119873, 0.7940094351724705, 0.6584725857227015, 0.6584725857227015, 0.6575289390269551, 0.6584725857227015, 0.6575289390269551, 0.6024621669702658, 0.5922350746642899, 0.5922350746642899, 0.5922350746642899, 0.7881858815834428, 0.7263308900881978, 0.7881858815834428, 0.8826462319410482, 0.7881858815834428, 0.7881858815834428, 0.70368417769669, 0.7263308900881978, 0.7881858815834428, 0.8826462319410482, 0.7881858815834428, 0.7881858815834428, 0.70368417769669, 0.7413701570157422, 0.7413701570157422, 0.7413701570157422]

 [

In [55]:
mover_results

{'de-en': 0.6}

Reproduction for the language pair 'cs-en'. The other 6 language pairs are still missing because of time constraints

In [56]:
mover_results = {}
better_scores = []
worse_scores = []
for lang_pair in ["cs-en"]:
    # load data
    wmt = WMT18(lang_pair, root="/tmp")

    for idx in range(len(wmt.translations_better)):
    # compute scores
        scores_better = mover()._compute(
            predictions = wmt.translations_better[idx],
            references = wmt.references[idx]
        )
        better_scores.append(scores_better)

        scores_worse = mover()._compute(
            predictions = wmt.translations_worse[idx],
            references = wmt.references[idx]
        )

        worse_scores.append(scores_worse)

    ks = kendall_score(better_scores, worse_scores)
    mover_results[lang_pair] = ks

In [57]:
mover_results

{'cs-en': 0.35459882583170255}

{'cs-en': 0.35459882583170255}